#### Import Dependencies & needed API Keys

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from time import sleep, time

from config import airNowApiKeyShane, airNowApiKeyAudelia, airNowApiKeyCenez, airNowApiKeyJoseph, airNowApiKeyJoey
from largest_combined_statistical_areas_v3_t25 import largest_combined_statistical_areas

# from daylight_savings_time_dates_v1 import daylight_savings_time_dates
from daylight_savings_time_dates_v1_2015_to_2020 import daylight_savings_time_dates

In [2]:
# largest_combined_statistical_areas
# daylight_savings_time_dates

#### Setup counters, empty list to hold data amd API Key Variables to be used

In [3]:
# Setup Counters
counterAttempts = 0
counterAttemptsPerAPI = 0
counterAPI = 1

# Setup empty list
apiData = []

# Setup API Key Variables
airNowApiKey1 = airNowApiKeyShane
airNowApiKey2 = airNowApiKeyAudelia
airNowApiKey3 = airNowApiKeyCenez
airNowApiKey4 = airNowApiKeyJoseph
airNowApiKey5 = airNowApiKeyJoey

#### Create a list of date ranges to iterate through (due to API limitations, list for each month: [monthStartDate, monthEndDate], January 2015 to April 2020)

In [4]:
# Set Date Range to use for API
startDate = "2015-01-01"
# startDate = "2020-01-01" # Used for testing
endDate = "2020-04-30"
# endDate = "2020-03-31" # Used for testing

# Create empty list to start
datesToUseList = []
monthStartDateList = pd.date_range(startDate,endDate, freq = '1M') - pd.offsets.MonthBegin(1)
monthEndDateList = pd.date_range(startDate,endDate, freq = '1M')

for date in monthEndDateList:
    datesToUseList.append([(date - pd.offsets.MonthBegin(1)).strftime("%Y-%m-%d"), date.strftime("%Y-%m-%d")])

# datesToUseList
len(datesToUseList)

64

#### Create a list of date that are within the Daylight Savings Time period to determine which UTC Offset to use for each city (January 2015 to Dec  2020)

In [6]:
# Create a date list of DST dates
dstDateList = []
for year in daylight_savings_time_dates:
    dstStart = year["dates"]["start"]
    dstEnd = year["dates"]["end"]
    dstDateListHolder = pd.date_range(dstStart, dstEnd).strftime("%Y-%m-%d").tolist()
    for date in dstDateListHolder:
        dstDateList.append(date)

# dstDateList
len(dstDateList)

1434

#### Setup API Call Function and with URL, API Key, and variables to pass through in the requests.get() API call

In [7]:
def apiAirNowObsByMonitoringSite(csaRank, csaName, csaPrimaryCity, csaPrimaryCityState, csaPopulation2018Estimate, csaPopulation2010Census, csaPrimaryCityLat, csaPrimaryCityLong, csaPrimaryCityZip, csaTimeZone, csaStandardTimeUtcOffset, csaDaylightSavingsTimeUtcOffset, csaMonitoringStationLat, csaMonitoringStationLong, csaSearchRadius, obsStartDate, obsStartHour, obsEndDate, ObsEndHour, apiKey, apiKeyUsed, counterAttempts, counterAttemptsPerAPI, counterAPI):
    # Empty response list variable every time function is called
    response = []
    
    # Set variables to pass through to API parameters
    particulates = "ozone,pm25,pm10,co,no2,so2"
#     particulates = "ozone,pm25,pm10"
    longMin = round(csaMonitoringStationLong - 0.10,3)
    longMax = round(csaMonitoringStationLong + 0.10,3)
    latMin = round(csaMonitoringStationLat - 0.10,3)
    latMax = round(csaMonitoringStationLat + 0.10,3)
    
    # Set API Parameters to be passed through to API requests.get
    params = {}
    params["baseURL"] = "http://www.airnowapi.org/aq/data/"
    params["obsStartDate"] = f"{obsStartDate}"
    params["obsStartHour"] = f"{obsStartHour}"
#     params["obsStartHour"] = "00"
    params["obsEndDate"] = f"{obsEndDate}"
    params["obsEndHour"] = f"{obsEndHour}"
#     params["obsEndHour"] = "23"
    params["particulates"] = f"{particulates}"
    params["bbox"] = f"{longMin},{latMin},{longMax},{latMax}"
    params["dataType"] = "B"
    params["format"] = "application/json"
    params["verbose"] = "1"
    params["nowCastOnly"] = "1"
    params["includeRawConcentrations"] = "1"
    params["apiKey"] = apiKey
    
    # Build API Request URL, passing through parameters - by GEOGRAPHIC BOUNDING BOX
    requestURL = params["baseURL"] \
                + "?startDate=" + params["obsStartDate"] + "T" + params["obsStartHour"] \
                + "&endDate=" + params["obsEndDate"] + "T" + params["obsEndHour"] \
                + "&parameters=" + params["particulates"] \
                + "&BBOX=" + params["bbox"] \
                + "&dataType=" + params["dataType"] \
                + "&format=" + params["format"] \
                + "&verbose=" + params["verbose"] \
                + "&nowcastonly=" + params["nowCastOnly"] \
                + "&includerawconcentrations=" + params["includeRawConcentrations"] \
                + "&API_KEY=" + params["apiKey"]
    
    # Logger: Print status message
    print("--------------------------------------------------")
    print("Requesting AirNow API Data for...")
    print("     **************************************************")
    print(f"CSA: {csaName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | minLatLong: [{latMin},{longMin}] | maxLatLong: [{latMax},{longMax}] | DATE RANGE: {obsStartDate} - {obsEndDate}")
    print(f"ATTEMPT:{counterAttempts} | ATTEMPT PER BATCH/API: {counterAttemptsPerAPI} | BATCH/API: {counterAPI} | API Key Used: {apiKeyUsed}")
    print("     **************************************************")
    
    # Set up error handling in the even there is an error in the API requests.get()
    try:
        
        # Execute requests.get, passing through built requestURL
        response = requests.get(requestURL).json()
        responseCode = requests.get(requestURL)
        
        # Results sum and response Code:
        print("     **************************************************")
        print(f"Number of results found: {len(response)}")
        print(f"Response Code: {responseCode}")
        print("     **************************************************")

        # Loop through response, appending each element (dictionary) as a new item in the apiData list
        for i in range(len(response)):
            
            # Add values brought in from the largest_combined_statistical_areas dictionary / json object to the response
            response[i]["csaRank"] = csaRank
            response[i]["csaName"] = csaName
            response[i]["csaPrimaryCity"] = csaPrimaryCity
            response[i]["csaPrimaryCityState"] = csaPrimaryCityState
            response[i]["csaPopulation2018Estimate"] = csaPopulation2018Estimate
            response[i]["csaPopulation2010Census"] = csaPopulation2010Census
            response[i]["csaPrimaryCityLat"] = csaPrimaryCityLat
            response[i]["csaPrimaryCityLong"] = csaPrimaryCityLong
            response[i]["csaPrimaryCityZip"] = csaPrimaryCityZip
          
            response[i]["csaTimeZone"] = csaTimeZone
            response[i]["csaStandardTimeUtcOffset"] = csaStandardTimeUtcOffset
            response[i]["csaDaylightSavingsTimeUtcOffset"] = csaDaylightSavingsTimeUtcOffset
            response[i]["csaMonitoringStationLat"] = csaMonitoringStationLat
            response[i]["csaMonitoringStationLong"] = csaMonitoringStationLong
            response[i]["csaSearchRadius"] = csaSearchRadius
            
            # Append the response to the apiData list
            apiData.append(response[i])
    
    except Exception as e:
        print("     **************************************************")
        print(f"Response Code: {responseCode}")
        print("     **************************************************")
        print(f"ERROR: Unable to perform AirNow API request for CSA: {csaName} | CITY: {csaPrimaryCity}, {csaPrimaryCityState} | minLatLong: [{latMin},{longMin}] | maxLatLong: [{latMax},{longMax}] | DATE RANGE: {obsStartDate} - {obsEndDate}")
        print("%s" % e)
        print("--------------------------------------------------")
        pass
    
    

#### Loop through cities and dates, while updating variables and passing them through to the apiAirNowObsByMonitoringSite Function

In [ ]:
# Loop through cities in the largest_combined_statistical_areas dictionary / json object
for csa in largest_combined_statistical_areas:
    
    # Loop through datesToUseList, calling the apiAirNowObsByMonitoringSite function, passing through variables
    for date in datesToUseList:
    
        # Set varaiable values to pass into the apiAirNowObsByMonitoringSite function
        csaRank = csa["csa_rank"]
        csaName = csa["csa_name"]
        csaPrimaryCity = csa["primary_city"]
        csaPrimaryCityState = csa["primary_city_state"]
        csaPopulation2018Estimate = csa["population"]["2018_estimate"]
        csaPopulation2010Census = csa["population"]["2010_census"]
        csaPrimaryCityLat = csa["primary_city_location"]["lat"]
        csaPrimaryCityLong = csa["primary_city_location"]["long"]
        csaPrimaryCityZip = csa["primary_city_location"]["zip_code"]
        csaTimeZone = csa["timezone_params"]["timezone"]
        csaStandardTimeUtcOffset = csa["timezone_params"]["utc_offset"]["standard_time"]
        csaDaylightSavingsTimeUtcOffset = csa["timezone_params"]["utc_offset"]["daylight_savings_time"]
        csaMonitoringStationLat = csa["search_params"]["closest_monitoring_station"]["lat"]
        csaMonitoringStationLong = csa["search_params"]["closest_monitoring_station"]["long"]
        csaSearchRadius = csa["search_params"]["closest_monitoring_station"]["search_radius"]
        
        obsStartDate = date[0]
        obsStartHour = "00"
        obsEndDate = date[1]
        obsEndHour = "23"
                
        # Update overall attempt counter
        counterAttempts += 1
        
        # Once 250 attempts have been made for all five API Keys, reset counterAttemptsPerAPI, counterAPI and apiKey then sleep for one hour (3600 seconds)
        if counterAttemptsPerAPI >= 250 and counterAPI >= 5:
            counterAttemptsPerAPI = 1
            counterAPI = 1
            apiKey = airNowApiKey1
            print("*****  250 per API for all APIs avialable reached | RESET counterAttemptsPerAPI, counterAPI, apiKey & SLEEP  *****")
            sleep(3600)
#             sleep(3)

        # Once 250 attempts have been made for the currently used API Key, update counterAPI so a new apiKey is used next time through, reset counterAttemptsPerAPI
        elif counterAttemptsPerAPI >= 250:
            counterAPI += 1
            counterAttemptsPerAPI = 1
            print("*****  250 per current API reached | RESET counterAttemptsPerAPI; UPDATE counterAPI & apiKey  *****")
            
        else:
            counterAttemptsPerAPI += 1

        # Set apiKey variable based on the counterAPI variable
        if counterAPI == 1:
            apiKey = airNowApiKey1
            apiKeyUsed = "airNowApiKey1"
        elif counterAPI == 2:
            apiKey = airNowApiKey2
            apiKeyUsed = "airNowApiKey2"
        elif counterAPI == 3:
            apiKey = airNowApiKey3
            apiKeyUsed = "airNowApiKey3"
        elif counterAPI == 4:
            apiKey = airNowApiKey4
            apiKeyUsed = "airNowApiKey4"
        elif counterAPI == 5:
            apiKey = airNowApiKey5
            apiKeyUsed = "airNowApiKey5"
        
        # Call the apiCallLatLongHistorical function, passing through updated variables as parameters
        apiAirNowObsByMonitoringSite(csaRank, csaName, csaPrimaryCity, csaPrimaryCityState, csaPopulation2018Estimate, csaPopulation2010Census, csaPrimaryCityLat, csaPrimaryCityLong, csaPrimaryCityZip, csaTimeZone, csaStandardTimeUtcOffset, csaDaylightSavingsTimeUtcOffset, csaMonitoringStationLat, csaMonitoringStationLong, csaSearchRadius, obsStartDate, obsStartHour, obsEndDate, ObsEndHour, apiKey, apiKeyUsed, counterAttempts, counterAttemptsPerAPI, counterAPI)

--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2015-01-01 - 2015-01-31
ATTEMPT:2 | ATTEMPT PER BATCH/API: 2 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 1483
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2015-02-01 - 2015-02-28
ATTEMPT:3 | ATTEMPT PER BATCH/API:

     **************************************************
Number of results found: 2693
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2016-03-01 - 2016-03-31
ATTEMPT:16 | ATTEMPT PER BATCH/API: 16 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2961
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined S

     **************************************************
Number of results found: 2822
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2017-04-01 - 2017-04-30
ATTEMPT:29 | ATTEMPT PER BATCH/API: 29 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 2499
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined S

     **************************************************
Number of results found: 1437
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2018-05-01 - 2018-05-31
ATTEMPT:42 | ATTEMPT PER BATCH/API: 42 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 1488
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined S

     **************************************************
Number of results found: 0
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Statistical Area | CITY: New York City, NY | minLatLong: [40.742,-73.936] | maxLatLong: [40.942,-73.736] | DATE RANGE: 2019-06-01 - 2019-06-30
ATTEMPT:55 | ATTEMPT PER BATCH/API: 55 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 988
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: New York-Newark, NY-NJ-CT-PA Combined Stati

...

     **************************************************
Number of results found: 1245
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statistical Area | CITY: Chicago, IL | minLatLong: [41.864,-87.759] | maxLatLong: [42.064,-87.559] | DATE RANGE: 2016-05-01 - 2016-05-31
ATTEMPT:146 | ATTEMPT PER BATCH/API: 146 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 1109
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Stati

     **************************************************
Number of results found: 1488
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statistical Area | CITY: Chicago, IL | minLatLong: [41.864,-87.759] | maxLatLong: [42.064,-87.559] | DATE RANGE: 2017-06-01 - 2017-06-30
ATTEMPT:159 | ATTEMPT PER BATCH/API: 159 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 1440
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Stati

     **************************************************
Number of results found: 714
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statistical Area | CITY: Chicago, IL | minLatLong: [41.864,-87.759] | maxLatLong: [42.064,-87.559] | DATE RANGE: 2018-07-01 - 2018-07-31
ATTEMPT:172 | ATTEMPT PER BATCH/API: 172 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 744
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statist

     **************************************************
Number of results found: 755
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statistical Area | CITY: Chicago, IL | minLatLong: [41.864,-87.759] | maxLatLong: [42.064,-87.559] | DATE RANGE: 2019-08-01 - 2019-08-31
ATTEMPT:185 | ATTEMPT PER BATCH/API: 185 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 1484
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Chicago-Naperville, IL-IN-WI Combined Statis

     **************************************************
Number of results found: 3986
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Washington-Baltimore-Arlington, DC-MD-VA-WV-PA Combined Statistical Area | CITY: Washington, DC | minLatLong: [38.819,-77.113] | maxLatLong: [39.019,-76.913] | DATE RANGE: 2015-05-01 - 2015-05-31
ATTEMPT:198 | ATTEMPT PER BATCH/API: 198 | BATCH/API: 1 | API Key Used: airNowApiKey1
     **************************************************
     **************************************************
Number of results found: 5658
Response Code: <Response [200]>
     **************************************************
--------------------------------------------------
Requesting AirNow API Data for...
     **************************************************
CSA: Washington-Baltimore-A

#### Check the number of records in the apiData list

In [ ]:
# apiData
len(apiData)

#### Loop through results and only use the records that are for a chosen datetime (noon local time for each city, checking to see if date is within Standard Time (ST) or Daylight Savings Time (DST) periods and then use the correct UTC offset

In [ ]:
# Set an empty list to hold the data being passed through
relevantApiData = []

# Loop through the apiData
for record in apiData:
    
    # Set variables and calculate desired local datetime for the record we want to use (12:00 noon, local time per each city)
    desiredLocalTime = "12:00"
    
    # Isolate just the date of the "UTC" datetime stamp string
    recordDate = pd.Timestamp(record["UTC"]).strftime("%Y-%m-%d")
    
    # Create variable for the city's local datetime we want to use
    desiredLocalDateTime = pd.Timestamp(f"{recordDate}T{desiredLocalTime}")

    # Create Standard Time variables to use
    stdTimeOffsetInt = int(record["csaStandardTimeUtcOffset"].replace("0","").replace(":",""))
    stdDesiredUTCDateTime = desiredLocalDateTime + timedelta(hours = (stdTimeOffsetInt * -1))
    stdDesiredUTCDateTimeStr = stdDesiredUTCDateTime.strftime("%Y-%m-%dT%H%:%M")
    
    # Create Daylight Savings Time variables to use
    dstTimeOffsetInt = int(record["csaDaylightSavingsTimeUtcOffset"].replace("0","").replace(":",""))
    dstDesiredUTCDateTime = desiredLocalDateTime + timedelta(hours = (dstTimeOffsetInt * -1))
    dstDesiredUTCDateTimeStr = dstDesiredUTCDateTime.strftime("%Y-%m-%dT%H%:%M")
    
    # If the record date is within a DST period and the "UTC" datetime stamp matches desired for DST
    if recordDate in dstDateList and record["UTC"] == dstDesiredUTCDateTimeStr:
        
        # Append the response to the apiData list
        record["st_dst"] = "DaylightSavings"
        record["obsDateTime"] = desiredLocalDateTime
        relevantApiData.append(record)
        
    # If the record date is within a ST period and the "UTC" datetime stamp matches desired for ST
    elif recordDate not in dstDateList and record["UTC"] == stdDesiredUTCDateTimeStr:
        
        # Append the response to the apiData list
        record["st_dst"] = "Standard"
        record["obsDateTime"] = desiredLocalDateTime
        relevantApiData.append(record)
        
    else:
        # Ignore the record, do not pass to relevantApiData
        pass
    
# relevantApiData
len(relevantApiData)

#### Create DataFrame holding the values from the apiData & relevantApiData lists

In [ ]:
apiResults_df = pd.DataFrame(apiData)
relevantApiResults_df = pd.DataFrame(relevantApiData)

# Visualize the DataFrame
apiResults_df
relevantApiResults_df



#### Set output data file filepath variable

In [ ]:
output_data_filepath = "output_data/"

#### Create variables to create the CSV file name

In [ ]:
timestamp = dt.now().strftime("%Y%m%d_%H%M%S")
output_file_name = f"AirNowAPI_{timestamp}"
# print(output_file_name)

#### Convert & export DataFrame to CSV and JSON files

In [ ]:
df.to_csv(f"{output_data_filepath}{output_file_name}.csv", encoding="utf-8", index= False)